In [1]:
# imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn

import platform
import logging
import sys
import os
from os import path

import warnings
import datetime as dt
from datetime import datetime

from statsmodels.tsa.arima_model import ARIMA

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# warnings.filterwarnings("ignore") # test

In [3]:
# Working Dir / Load Data
from sys import platform
# if platform.system() == 'Windows':
if platform == 'win32':
    directory = 'D:\\project\\data\\kg_jpn_rest\\'
    exportDirectory = directory + 'export\\'

# Mac
#elif platform.system() == 'Darwin':
elif platform == 'darwin':
    directory = '//Project/data/kg_corpgroc/'
    exportDirectory = directory + 'export/'

# AWS
elif platform == 'linux':
    directory = '//data/'
    exportDirectory = directory + 'export/'
    

In [4]:
GLOBAL_CONST_DEBUG = 0

## Load Data

In [5]:
# Code to load notebooks borrowed from online
# http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]   
    
sys.meta_path.append(NotebookFinder())

In [6]:
import helper_notebook as hlp

In [7]:
# Load the data set / cleaned, joined, and formatted
dfSuper = hlp.fn_load_all_data(1)

Data Load - Loading data on: Windows Directory: D:\project\data\kg_jpn_rest\
0:00:10.643466
Data Load - Merging Data
0:01:00.952805
Data Load - Adding Features
0:01:13.945001
Data Load - Finished


In [8]:
dfSuper.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 252108 entries, 2016-01-13 to 2017-04-22
Data columns (total 48 columns):
air_store_id        252108 non-null object
visitors            252108 non-null float32
calendar_date       252108 non-null datetime64[ns]
day_of_week         252108 non-null object
holiday_flg         252108 non-null int64
genre_name	         252108 non-null object
area_name           252108 non-null object
latitude            252108 non-null float64
longitude           252108 non-null float64
reserve_visitors    252108 non-null float64
month_num           252108 non-null int64
month_name          252108 non-null object
dayofmonth_num      252108 non-null int64
dayofweek_num       252108 non-null int64
weekend             252108 non-null int64
corr_vis_resv       252108 non-null float64
cycle               252108 non-null float64
trend               252108 non-null float64
SMA_3_days          252108 non-null float64
SMA_7_days          252108 non-null float64
S

In [9]:
dfSuper.head()

,air_store_id,visitors,calendar_date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,reserve_visitors,...,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,forecast
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-01-13,air_ba937bf13d40fb24,25.0,2016-01-13,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-14,air_ba937bf13d40fb24,32.0,2016-01-14,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-15,air_ba937bf13d40fb24,29.0,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-16,air_ba937bf13d40fb24,22.0,2016-01-16,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-18,air_ba937bf13d40fb24,6.0,2016-01-18,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0


In [10]:


# reservations in the air system
pd_air_reserve = pd.read_csv(directory + 'air_reserve.csv', parse_dates=(['visit_datetime','reserve_datetime']), infer_datetime_format=True)
pd_air_reserve.name = 'pd_air_reserve'
#•	air_store_id - the restaurant's id in the air system
#•	visit_datetime - the time of the reservation
#•	reserve_datetime - the time the reservation was made
#•	reserve_visitors - the number of visitors for that reservation

# reservation in the hpg system
pd_hpg_reserve = pd.read_csv(directory + 'hpg_reserve.csv', parse_dates=(['visit_datetime','reserve_datetime']), infer_datetime_format=True)
pd_hpg_reserve.name = 'pd_hpg_reserve'
#•	hpg_store_id - the restaurant's id in the hpg system
#•	visit_datetime - the time of the reservation
#•	reserve_datetime - the time the reservation was made
#•	reserve_visitors - the number of visitors for that reservation

# contains info about the store info.  lat and long is the area which the store belongs
pd_air_store_info = pd.read_csv(directory + 'air_store_info.csv', infer_datetime_format=True)
pd_air_store_info.name = 'pd_air_store_info'
#•	air_store_id
#•	air_genre_name
#•	air_area_name
#•	latitude
#•	longitude

# contains info about select air restaraunts.  Lat and long is the area where store belongs
pd_hpg_store_info = pd.read_csv(directory + 'hpg_store_info.csv', infer_datetime_format=True)
pd_hpg_store_info.name = 'pd_hpg_store_info'
#•	hpg_store_id
#•	hpg_genre_name
#•	hpg_area_name
#•	latitude
#•	longitude

# file contains HISTORICAL visit data for the air restaraunts
pd_air_visit_data = pd.read_csv(directory + 'air_visit_data.csv', parse_dates=(['visit_date']), infer_datetime_format=True)
pd_air_visit_data.name = 'pd_air_visit_data'
#•	air_store_id
#•	visit_date - the date
#•	visitors - the number of visitors to the restaurant on the date

# give basic info about the calendar dates in the dataset
pd_date_info = pd.read_csv(directory + 'date_info.csv', parse_dates=(['calendar_date']), infer_datetime_format=True)
pd_date_info.name = 'pd_date_info'

# allows you to join select restaraunts that have both air and hpg systems
pd_store_id_relation = pd.read_csv(directory + 'store_id_relation.csv', infer_datetime_format=True)
pd_store_id_relation.name = 'pd_store_id_relation'
#•	hpg_store_id
#•	air_store_id



pd_sample_submission = pd.read_csv(directory + 'sample_submission.csv')

## Prepare Data

In [11]:
# Set Indexes
pd_air_visit_data.set_index('visit_date', inplace=True)
pd_date_info.set_index('calendar_date', inplace=True)

In [12]:
# Set data types
pd_air_visit_data['visitors'] = pd_air_visit_data['visitors'].astype('float32')
pd_air_visit_data['visitors_log'] = np.log(pd_air_visit_data['visitors'])

# Create columns and default (initialize)
pd_air_visit_data['forecast'] = 0
pd_air_visit_data['forecast_log'] = 0
pd_air_visit_data['forecast_logExp'] = 0

In [13]:
# Join Holidays / Reservations List
# TODO: Implement
visitMergeDF = pd_air_visit_data.copy()
# 01/15/2018 - Ben Grauer - changed over to new style
visitMergeDF = dfSuper.copy()

In [14]:
visitMergeDF.head()

,air_store_id,visitors,calendar_date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,reserve_visitors,...,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,forecast
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-01-13,air_ba937bf13d40fb24,25.0,2016-01-13,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-14,air_ba937bf13d40fb24,32.0,2016-01-14,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-15,air_ba937bf13d40fb24,29.0,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-16,air_ba937bf13d40fb24,22.0,2016-01-16,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0
2016-01-18,air_ba937bf13d40fb24,6.0,2016-01-18,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0.0,...,0,1,0,0,0,0,0,0,0,0.0


## TESTING  

In [15]:
testing = 0

In [16]:
if testing==1:
    testTS.head(3)

In [17]:
testing = 0
if testing == 1:
    
    # Make a copy
    #testTS = dfSuper.copy()
    
    storeToEvaluate = 'air_1c0b150f9e696a5f'
    storeToEvaluate = 'air_55390f784018349a'

    # make a copy
    #testTS = dfSuper[dfSuper['air_store_id']==storeToEvaluate].copy()
    testTS = dfSuper.loc[(dfSuper.index.get_level_values('air_store_id') == storeToEvaluate)]

    # Reset just to the visit date
    testTS.reset_index(level=0, inplace=True)
    
    testTS.asfreq('D')
    print('done')

    testTS['visitors_log'] = np.log(testTS['visitors'])


    # ts.index = ts.index + pd.DateOffset(days=39)  # this was populating data?
    # Re-initialize variables for function below  - 39 days
    idx = pd.DataFrame(pd.date_range('2017-04-23','2017-05-31'), columns={'dateRange'})
    idx.set_index('dateRange',inplace=True)
    testTS = pd.concat([testTS,idx], axis=1)

    # Impute
    testTS['air_store_id'].replace({np.nan: storeToEvaluate}, inplace=True)
    testTS['visitors'].replace({np.nan: 0}, inplace=True)
    testTS['visitors_log'].replace({np.nan: 0}, inplace=True)

In [18]:
#minDate = ts.index.min
#print(minDate)
#print('\n')

# ts["2017-04-01":"2017-04-23"]
#ts[ts['visitors_log']==np.nan]

#print(len(ts))
#print(len(ts.dropna()))
#ts.head(51)
#testTS.head()

In [19]:
if testing == 1:
    # Find Min Date
    minDate = testTS.index.min()

    testTS['visitors'][minDate:"2017-04-23"].plot()
    testTS['visitors_log'][minDate:"2017-04-23"].plot()

In [21]:
def fn_run_TS_test(colName, modelName, tsParam):
    minDate = testTS.index.min()

    model = ARIMA(testTS['visitors'][minDate:"2017-04-22"], order=(tsParam))
    modelFit = model.fit()
    results = modelFit.forecast(39)
    # Create a couple of new fiels to see the forecast
    #strModelName = 'forecast_' + modelName
    #print(strModelName)

    #armRsltDF['id'] = armRsltDF['id'].str.cat(armRsltDF['air_store_id'])
    #armRsltDF['id'] = armRsltDF['id'] + '_'
    #armRsltDF['id'] = armRsltDF['id'].str.cat(armRsltDF['visit_date'].astype(str))

    testTS[modelName] = 0
    testTS[modelName]["2017-04-23":"2017-05-31"] = results[0][:]

In [22]:
if testing == 1:
    warnings.filterwarnings("ignore")

    fn_run_TS_test('visitors', 'forecast_7day_7,0,0', (7,0,0))
    fn_run_TS_test('visitors', 'forecast_7day_7,1,0', (7,1,0))
    fn_run_TS_test('visitors', 'forecast_14day_14,0,0', (14,0,0))
    fn_run_TS_test('visitors', 'forecast_14day_14,1,0', (14,1,0))
    #fn_run_TS_test('visitors', 'forecast_7day_7,1,0', (15,1,0))
    fn_run_TS_test('visitors', 'forecast_14day_14,1,0', (14,1,0))

In [23]:
if testing == 1:
    startTime = datetime.now()

    #do something

    # this looks alot better
    fn_run_TS_test('31-day-SMA', 'forecast_31SMA_0,0,0', (31,0,0))


    print(datetime.now() - startTime)
    testTS[['visitors','forecast_31SMA_0,0,0']]["2017-01-01":"2017-05-31"].plot()

In [24]:
if testing == 1:
    from datetime import datetime
    startTime = datetime.now()

    #do something

    # this looks alot better
    fn_run_TS_test('7-day-SMA', 'forecast_7SMA_0,0,0', (31,0,0))

    testTS[['visitors','forecast_7SMA_0,0,0']]["2017-01-01":"2017-05-31"].plot()       

    print (datetime.now() - startTime) # about 13 minutes

In [25]:
if testing == 1:
    finalList = ['visitors','forecast_7day_7,0,0','forecast_7day_7,1,0','forecast_14day_14,0,0','forecast_14day_14,1,0',
                '7-day-SMA','14-day-SMA','31-day-SMA']
    testTS[finalList]["2017-01-01":"2017-05-31"].plot()

In [26]:
if testing == 1:
    #ts['forecast']["2017-04-23":"2017-05-31"] = results_7day[0][:]
    #ts['forecast_log']["2017-04-23":"2017-05-31"] = resultsLog[0][:]
    #ts['forecast_logExp'] = np.exp(ts['forecast_log'])  # Return Log back
    #ts.plot()
    #print(modelFit_7day.aic)

    testTS['forecast']["2017-04-23":"2017-05-31"] = results_15day[0][:]
    testTS['forecast_log']["2017-04-23":"2017-05-31"] = resultsLog[0][:]
    testTS['forecast_logExp'] = np.exp(testTS['forecast_log'])  # Return Log back
    testTS.plot()
    print(modelFit_15day.aic)
    print(modelLog_fit.aic)

## END TESTING

## BEGIN LOOP

###   Functions

### Main Code Bock Start

In [27]:
# Variable to specify if we include a header one time in the files
includeHeaderRunOnce = True
# Determine if we are resuming a previous file
resumeRunningPreviousFile = False


#==================
# Setup Directories
#==================
# if platform.system() == 'Windows':
if platform == 'win32':
    exportDirectory = directory + 'export\\'

# Mac
#elif platform.system() == 'Darwin':
elif platform == 'darwin':
    exportDirectory = directory + 'export/'

# AWS
elif platform == 'linux':
    exportDirectory = directory + 'export/'

#exportParamOptionsFileName = exportDirectory + 'export_param_' + str(file_args_store_nbr) + '.csv'
exportResultsSubmissionFileName = exportDirectory + 'ARIMA_export_results.csv'

exportLogName = exportDirectory + 'ARIMA_export_log.log'


# ===========================
# SETUP LOGGING
# ===========================
# Wipe any existing log file - change to keep this script
#if path.isfile(exportLogName):
#    os.remove(exportLogName)

# We may set another parameter to pass in to wipe the existing param options and results submissions

# SEt logging information
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# create a file handler
handler = logging.FileHandler(exportLogName)
handler.setLevel(logging.INFO)

# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(handler)
logger.info('Start Logging')


# Find out if we have an existing file and work with it from there to update the mergedDataFrame
includeSubmissionHeaderRunOnce = True

In [28]:
# Set a default running dataframe - to be used to grab unique values
# ALWAYS NEEDS TO BE HERE
rundf = visitMergeDF.copy()

### Check if we are resuming File

In [29]:
# FUNCTIONS
# Check if the file exists
def fn_determine_file_exists(fileName):
    fileExists = False

    if path.isfile(fileName):
        fileExists = True

    return  fileExists

# Will determine where the file / process left off to pick back up
def fn_determine_file_last_run(fileName, colNames):

    # Read In
    df_leftOff = pd.read_csv(fileName)
    
    # Grab the unique store number 
    df_leftOff = pd.DataFrame(df_leftOff['air_store_id'].unique(), columns=(colNames))
    
    # if I re-use this later, need to handle for multiple column names (if applicable)
    df_leftOff.sort_values(['air_store_id'], ascending=[True], inplace=True)

    # Set a Processed Flag for all the entries
    df_leftOff['processed'] = 1

    # Return a data frame to join later
    return df_leftOff

In [30]:
# Determine if we are resuming a file
# this could stand to be cleaned up.  

# Call the function to determine where we left off
if fn_determine_file_exists(exportResultsSubmissionFileName) == True:

    # Repull
    existingDF = fn_determine_file_last_run(exportResultsSubmissionFileName, ['air_store_id'])
    print('Existing File detected with ' + str(len(existingDF)) + ' entries.')

    # Join
    resumeDF = pd.merge(rundf.reset_index(), existingDF, how='left', on='air_store_id', copy=True)
    resumeDF['processed'].replace({np.nan: 0}, inplace=True)
    resumeDF = resumeDF[resumeDF['processed']==0].copy()
    resumeDF.set_index('visit_date', inplace=True)
    
    # Drop the processed flag here - future (or keep if need be)
    
    # re order
    resumeDF.sort_values(['air_store_id'], ascending=[True], inplace=True)

    # set to True for below
    resumeRunningPreviousFile = True
    
    # Set the runnign dataframe to the resumed one
    rundf = resumeDF.copy()

# Determine if we include header (multiple booleans as I had split multiple output files earlier)
if resumeRunningPreviousFile == True:
    includeSubmissionHeaderRunOnce = False
    
# Re-Order - whether we are resuming or not
rundf.sort_values(['air_store_id'], ascending=[True], inplace=True)


Existing File detected with 483 entries.


In [31]:
rundf.head()

,air_store_id,visitors,calendar_date,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,reserve_visitors,...,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,forecast,processed
visit_date,,,,,,,,,,,,,,,,,,,,,
2017-03-08,air_96743eee94114261,22.0,2017-03-08,Wednesday,0,Japanese food,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,37.916367,139.036327,19.0,...,0,0,0,1,0,0,0,0,0.0,0.0
2016-02-01,air_96743eee94114261,4.0,2016-02-01,Monday,0,Japanese food,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,37.916367,139.036327,0.0,...,0,0,0,0,0,0,0,0,0.0,0.0
2016-01-30,air_96743eee94114261,23.0,2016-01-30,Saturday,0,Japanese food,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,37.916367,139.036327,17.0,...,1,0,0,0,0,0,0,0,0.0,0.0
2016-01-05,air_96743eee94114261,2.0,2016-01-05,Tuesday,0,Japanese food,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,37.916367,139.036327,0.0,...,1,0,0,0,0,0,0,0,0.0,0.0
2016-01-06,air_96743eee94114261,12.0,2016-01-06,Wednesday,0,Japanese food,Niigata-ken Niigata-shi Gakkōchōdōri 1 Banchō,37.916367,139.036327,3.0,...,1,0,0,0,0,0,0,0,0.0,0.0


### Loop through all records

In [32]:
def fn_run_arima_timeseries(storeToEvaluate, orderList):
    warnings.filterwarnings("ignore")
    errorOccured = False
    
    if GLOBAL_CONST_DEBUG ==1:
        print('Start Processing Restaraunt: ' + str(storeToEvaluate))
    
    # create ts sub-set
    ts = visitMergeDF[visitMergeDF['air_store_id']==storeToEvaluate].copy()
    ts.asfreq('D')
    
    minDate = ts.index.min()
    maxDate = ts.index.max()
    
    # add dates to predict
    idx = pd.DataFrame(pd.date_range('2017-04-23','2017-05-31'), columns={'dateRange'})
    idx.set_index('dateRange',inplace=True)
    ts = pd.concat([ts,idx], axis=1)

    # Impute
    ts['air_store_id'].replace({np.nan: storeToEvaluate}, inplace=True)
    ts['visitors'].replace({np.nan: 0}, inplace=True)
    # 01/15/2018 - Ben Grauer - took out
    #ts['visitors_log'].replace({np.nan: 0}, inplace=True)
    
    # Try Catch Here
    try:
    
        # standard ARIMA model with order list passed in
        # 01/15/2018 - Ben Grauer - change from 23 to 22
        model = ARIMA(ts['visitors'][minDate:"2017-04-22"], order=orderList)
        modelFit = model.fit(disp=-1)
        # 39 days is how far out we are predicting
        results = modelFit.forecast(39)  
        # set forecast
        ts['forecast']["2017-04-23":"2017-05-31"] = results[0][:]

        if GLOBAL_CONST_DEBUG ==1:
            print('Start Processing Restaraunt - LOG: ' + str(storeToEvaluate))

        # 01/15/2018 - Ben Grauer - remove log
        # Do the log while we are in here
        #modelLog = ARIMA(ts['visitors_log'][minDate:"2017-04-23"], order=orderList)
        #modelLog_fit = modelLog.fit(disp=-1)
        #resultsLog = modelLog_fit.forecast(39)
        #ts['forecast_log']["2017-04-23":"2017-05-31"] = resultsLog[0][:]
        #ts['forecast_logExp'] = np.exp(ts['forecast_log'])  # Revert log back to standard
    
    
    except: 
        #(RuntimeError, TypeError, NameError):
        #print('Error')
        #print(RuntimeError)
        #print(TypeError)
        #print(NameError)
        #print('\n')
        logger.info('Error processing store: ' + storeToEvaluate)
        errorOccured = True
        
        if str(maxDate) == "2017-04-22 00:00:00":
            smaSevenDays = ts['SMA_7_days']["2017-04-22":"2017-04-22"][0]
            #dfTSReturn = ts[:]["2017-04-23":"2017-05-31"]
            ts['forecast'] = smaSevenDays
        else:
            # restaraunt has likely shut down
            ts['forecast'] = 0
           
    pass


    if GLOBAL_CONST_DEBUG ==1:
        print('Finished Processing Restaraunt: ' + str(storeToEvaluate))


    # return back only what we predicted
    dfTSReturn = ts[:]["2017-04-23":]
    return dfTSReturn, errorOccured

In [33]:
# Get all Unique Visits
visitStoreArr = rundf['air_store_id'].unique()
warnings.filterwarnings("ignore")
# 12/28/2017 - This is the first order list I want to go with
# orderList = (15,1,0)
# 12/29/2017 - This is the second list (really want the log values)
# orderList = (20,1,0)
# 01/15/2018 - Attemp 31 - test with 3
orderList = (31,0,0)

# print(orderList)

# Default to the max of the array
maxLoopRun = len(visitStoreArr)

# if we are testing
testRun = 0
if testRun > 0:
    maxLoopRun = testRun
    includeSubmissionHeaderRunOnce = True
    # Test an error
    #visitStoreArr = ('air_1c0b150f9e696a5f')

print('Total Records to Process: ' + str(maxLoopRun))
startTime = datetime.now()

i=0
while i < maxLoopRun: 
    
    #if i%300==0:
    if i%300==0:
        print('Processing Index: ' + str(i) + ' - Store ID: ' + visitStoreArr[i] + 
              ' - Elapsed Time: ' + str(datetime.now() - startTime))
    
    # Log Start
    logger.info('Start Index: ' + str(i) + ' - Restaraunt: ' + str(visitStoreArr[i]))
    
    # Run TS
    dfTsRun, bError = fn_run_arima_timeseries(visitStoreArr[i], orderList)
    
    # handle error in side for this script
    # if bError == True:

        
    
    # TODO:Reformat (possibly done), any numbers
    dfTsRun.reset_index(inplace=True)
    dfTsRun = dfTsRun[['index','air_store_id','visitors','forecast']]
    # colNames = ('visit_date','air_store_id','visitors','visitors_log','forecast','forecast_log','forecast_logExp')
    dfTsRun = dfTsRun.rename(columns={'index': 'visit_date'})
    
    with open(exportResultsSubmissionFileName, 'a') as f:
        dfTsRun.to_csv(f, header=includeSubmissionHeaderRunOnce, index=False, quotechar='"')
        f.close()
        includeSubmissionHeaderRunOnce = False
    
    # Log Start
    logger.info('Finish Index: ' + str(i) + ' - Restaraunt: ' + str(visitStoreArr[i]))
    
    # increment
    i=i+1
    
print('\n')
print('Finished All Records')
print ('Time to Finish: ' + str(datetime.now() - startTime)) 

Total Records to Process: 346
Processing Index: 0 - Store ID: air_96743eee94114261 - Elapsed Time: 0:00:00
Processing Index: 300 - Store ID: air_f068442ebb6c246c - Elapsed Time: 1 day, 22:31:41.607961


Finished All Records
Time to Finish: 2 days, 5:32:50.782839


In [34]:
# dfTsRun.head()

In [35]:
# A Final step to massage the output file

In [36]:
logger.info('Stop Logging')
handlers = logger.handlers
for handler in handlers:
    handler.close()

## SUBMISSION FILES

In [37]:
# pull the results and formulate files
# Must have exact number in sample submission.  Not what all is in train. 

# pd_sample_submission

In [38]:
arimaExportDir = 'D:\\project\\data\\kg_jpn_rest\\export\\'
arimaExportFile = 'ARIMA_export_results.csv'

In [39]:
armRsltDF = pd.read_csv(arimaExportDir + arimaExportFile)

In [40]:
print(str(len(armRsltDF)))
armRsltDF.head()

32331


,visit_date,air_store_id,visitors,forecast
0,2017-04-23,air_00a91d42b08b08d9,0.0,15.228411
1,2017-04-24,air_00a91d42b08b08d9,0.0,25.130676
2,2017-04-25,air_00a91d42b08b08d9,0.0,25.105141
3,2017-04-26,air_00a91d42b08b08d9,0.0,32.591370
4,2017-04-27,air_00a91d42b08b08d9,0.0,31.828432


In [41]:
# Submission Format
#[Store]_[YYYY-MM-DD],[forecast]
# id,visitors
# air_00a91d42b08b08d9_2017-04-23,0

# Errors with these, they were included int he file
ErrorList = ('air_1c0b150f9e696a5f','air_7420042ff75f9aca',
             'air_789103bf53b8096b','air_900d755ebd2f7bbd',
             'air_a17f0778617c76e2','air_a9a380530c1e121f')

In [42]:
# Concatenate the fields to create the ID
armRsltDF['id'] = ''
armRsltDF['id'] = armRsltDF['id'].str.cat(armRsltDF['air_store_id'])
armRsltDF['id'] = armRsltDF['id'] + '_'
armRsltDF['id'] = armRsltDF['id'].str.cat(armRsltDF['visit_date'].astype(str))

# Drop the visitors (will rename later)
armRsltDF.drop('visitors', axis=1, inplace=True)

In [43]:
armRsltDF.head()

,visit_date,air_store_id,forecast,id
0,2017-04-23,air_00a91d42b08b08d9,15.228411,air_00a91d42b08b08d9_2017-04-23
1,2017-04-24,air_00a91d42b08b08d9,25.130676,air_00a91d42b08b08d9_2017-04-24
2,2017-04-25,air_00a91d42b08b08d9,25.105141,air_00a91d42b08b08d9_2017-04-25
3,2017-04-26,air_00a91d42b08b08d9,32.591370,air_00a91d42b08b08d9_2017-04-26
4,2017-04-27,air_00a91d42b08b08d9,31.828432,air_00a91d42b08b08d9_2017-04-27


In [44]:
# Must turn any negatives into a zero
# Not sure why this needs to come after, perhaps with the merge it throws some things between float/int offf
def fn_set_negative_to_zero(col):
    if col < 0:
        return 0.0
    else:
        return col

# 01/15/2018 - Ben Grauer - only use forecast
armRsltDF['forecast'] = armRsltDF.apply(lambda row: fn_set_negative_to_zero(row['forecast']), axis=1)
#armRsltDF['forecast_log'] = armRsltDF.apply(lambda row: fn_set_negative_to_zero(row['forecast_log']), axis=1)
#armRsltDF['forecast_logExp'] = armRsltDF.apply(lambda row: fn_set_negative_to_zero(row['forecast_logExp']), axis=1)

#armRsltDF['forecast_log'] = armRsltDF['forecast_log'].replace(np.nan, 0, regex=True)
#armRsltDF['forecast_logExp'] = armRsltDF['forecast_logExp'].replace(np.nan, 0, regex=True)


#armRsltDF[armRsltDF['forecast']<0]['forecast'] = 0.0
#armRsltDF[armRsltDF['forecast_log']<0]['forecast_log'] = 0.0
#armRsltDF[armRsltDF['forecast_logExp']<0]['forecast_logExp'] = 0.0

# do some data massaging
#armRsltDF['forecast_rnd'] = np.round(armRsltDF['forecast'])
#armRsltDF['forecast_logExp_rnd'] = np.round(armRsltDF['forecast_logExp'])

In [45]:
# Join to sample submission
armRsltDF = pd.merge(pd_sample_submission, armRsltDF, how='inner', on=('id'))

In [46]:
# Drop the visitors (will rename later)
armRsltDF.drop('visitors', axis=1, inplace=True)

# 32019 - correct length
len(armRsltDF)

32019

In [47]:
armRsltDF.head()

,id,visit_date,air_store_id,forecast
0,air_00a91d42b08b08d9_2017-04-23,2017-04-23,air_00a91d42b08b08d9,15.228411
1,air_00a91d42b08b08d9_2017-04-24,2017-04-24,air_00a91d42b08b08d9,25.130676
2,air_00a91d42b08b08d9_2017-04-25,2017-04-25,air_00a91d42b08b08d9,25.105141
3,air_00a91d42b08b08d9_2017-04-26,2017-04-26,air_00a91d42b08b08d9,32.591370
4,air_00a91d42b08b08d9_2017-04-27,2017-04-27,air_00a91d42b08b08d9,31.828432


In [48]:
def fn_write_submission_file(df, colNames, fileName):
    
    print(colNames[1])
    forecastCol = colNames[1]
    
    tempDF = df.copy()
    
    # Rename the second column to "visitors" as per submission
    tempDF.rename(columns={forecastCol: 'visitors'}, inplace=True)
    
    #print(tempDF.head())
    tempDF.to_csv(fileName, header=True, index=False, quotechar='"', columns=('id','visitors'))
    print('Wrote file: ' + fileName)

In [49]:
todayDate = str(dt.date.today().strftime('%Y%m%d'))

# Float forecast
exportDF = armRsltDF.copy()
exportDF
columns=('id','forecast')
fn_write_submission_file(armRsltDF, columns, arimaExportDir + todayDate + 'ARIMA_subm_frcst.csv')

# Rounded Forecast
#columns=('id','forecast_rnd')
#fn_write_submission_file(armRsltDF, columns, arimaExportDir + todayDate + '_subm_frcst_rnd.csv')

# Log Forecast
#columns=('id','forecast_logExp')
#fn_write_submission_file(armRsltDF, columns, arimaExportDir + todayDate + '_subm_frcst_log_flt.csv')

# Log Rounded Forecast
#columns=('id','forecast_logExp_rnd')
#fn_write_submission_file(armRsltDF, columns, arimaExportDir + todayDate + '_subm_frcst_log_rnd.csv')


forecast
Wrote file: D:\project\data\kg_jpn_rest\export\20180124ARIMA_subm_frcst.csv


## TROUBLESHOOTING INCORRECT LENGTH

In [50]:
'''
Fixed

# need to have 32,019  rows
len(armRsltDF)
# I have 32,331, which is 312 too many.  

armRsltDF['count1'] = 1

pivotResults = armRsltDF.pivot_table(values=['count1'],
                                       index=['air_store_id'],
                                       aggfunc={'count1':np.sum})

pivotResults.head()

pivotResults.sort_values(['count1'], ascending=[False], inplace=True)

pivotResults.head()

armRsltDF[armRsltDF['id']=='air_789103bf53b8096b_2017-04-23']
'''

"\n# need to have 32,019  rows\nlen(armRsltDF)\n# I have 32,331, which is 312 too many\n\narmRsltDF['count1'] = 1\n\npivotResults = armRsltDF.pivot_table(values=['count1'],\n                                       index=['air_store_id'],\n                                       aggfunc={'count1':np.sum})\n\npivotResults.head()\n\npivotResults.sort_values(['count1'], ascending=[False], inplace=True)\n\npivotResults.head()\n\narmRsltDF[armRsltDF['id']=='air_789103bf53b8096b_2017-04-23']\n"